In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [2]:
reg_data = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [3]:
ncaa_data = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19


In [4]:
#removing WLoc
reg_data['WLoc'].value_counts()

H    63049
A    33019
N    10766
Name: WLoc, dtype: int64

In [5]:
ncaa_data['WLoc'].value_counts()
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

N    1248
Name: WLoc, dtype: int64

In [6]:
# thinking of removing NumOT
print("Regular Season\n",reg_data['NumOT'].value_counts())
print("NCAA\n",ncaa_data['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    100790
1      5004
2       844
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [7]:
drop_cols = ['WLoc']
reg_data = reg_data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'NumOT' 'WFGM'
 'WFGA' 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl'
 'WBlk' 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR'
 'LAst' 'LTO' 'LStl' 'LBlk' 'LPF']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [8]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum
    new_reg_data['NumOT'] = row.NumOT

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        try:
            new_reg_data['FGP_A'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.WOR
        new_reg_data['DR_A'] = row.WDR
        new_reg_data['AST_A'] = row.WAst
        new_reg_data['TO_A'] = row.WTO
        new_reg_data['STL_A'] = row.WStl
        new_reg_data['BLK_A'] = row.WBlk
        new_reg_data['PF_A'] = row.WPF
        new_reg_data['B'] = row.LTeamID
        try:
            new_reg_data['FGP_B'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.LOR
        new_reg_data['DR_B'] = row.LDR
        new_reg_data['AST_B'] = row.LAst
        new_reg_data['TO_B'] = row.LTO
        new_reg_data['STL_B'] = row.LStl
        new_reg_data['BLK_B'] = row.LBlk
        new_reg_data['PF_B'] = row.LPF
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        try:
            new_reg_data['FGP_A'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.LOR
        new_reg_data['DR_A'] = row.LDR
        new_reg_data['AST_A'] = row.LAst
        new_reg_data['TO_A'] = row.LTO
        new_reg_data['STL_A'] = row.LStl
        new_reg_data['BLK_A'] = row.LBlk
        new_reg_data['PF_A'] = row.LPF
        new_reg_data['B'] = row.WTeamID
        try:
            new_reg_data['FGP_B'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.WOR
        new_reg_data['DR_B'] = row.WDR
        new_reg_data['AST_B'] = row.WAst
        new_reg_data['TO_B'] = row.WTO
        new_reg_data['STL_B'] = row.WStl
        new_reg_data['BLK_B'] = row.WBlk
        new_reg_data['PF_B'] = row.WPF
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


106834it [00:01, 61394.86it/s]


,Season,DayNum,NumOT,A,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,B,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,Win
0,2003,10,0,1104,46.551724,21.428571,61.111111,14,24,13,23,7,1,22,1328,41.509434,20.000000,72.727273,10,22,8,18,9,2,20,1
1,2003,10,0,1393,35.820896,25.000000,45.000000,20,25,7,12,8,6,16,1272,41.935484,40.000000,52.631579,15,28,16,13,4,4,18,0
2,2003,11,0,1437,30.136986,11.538462,60.869565,31,22,9,12,2,5,23,1266,41.379310,44.444444,58.620690,17,26,15,10,5,2,25,0
3,2003,11,0,1457,36.734694,27.272727,53.333333,17,20,9,19,4,3,23,1296,47.368421,33.333333,54.838710,6,19,11,12,14,2,18,0
4,2003,11,0,1208,38.709677,37.500000,62.962963,21,15,12,10,7,1,14,1400,49.180328,42.857143,84.615385,17,22,12,14,4,4,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,0,1403,61.702128,58.333333,64.285714,5,31,11,16,4,1,17,1328,35.593220,27.272727,80.000000,6,13,13,8,10,1,12,1
106830,2023,113,0,1103,44.067797,19.047619,50.000000,9,23,10,10,3,0,11,1405,52.459016,50.000000,76.923077,7,25,16,7,7,7,17,0
106831,2023,113,0,1429,41.176471,25.925926,72.727273,7,27,11,12,7,3,15,1461,31.481481,26.086957,75.000000,7,25,6,11,6,3,19,1
106832,2023,113,0,1433,52.542373,57.894737,83.333333,8,23,14,9,6,5,16,1386,37.500000,28.571429,72.222222,5,20,10,11,3,3,15,1


In [9]:
new_reg_data['Win'].value_counts()

1    53499
0    53335
Name: Win, dtype: int64

In [10]:
new_reg_data.corr()

,Season,DayNum,NumOT,A,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,B,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,Win
Season,1.000000,-0.048352,-0.007047,0.009737,0.009627,-0.018803,0.061249,-0.229456,0.096219,-0.031613,-0.194426,-0.091840,-0.003109,-0.197884,0.009921,0.006657,-0.024972,0.074298,-0.228794,0.096140,-0.036327,-0.195527,-0.087865,-0.001165,-0.197023,-0.002958
DayNum,-0.048352,1.000000,0.029183,0.000967,0.023299,0.021665,0.051120,-0.057101,-0.039360,-0.031691,-0.142821,-0.061059,-0.026456,-0.034333,0.001270,0.021956,0.022861,0.051968,-0.047604,-0.038431,-0.032322,-0.142000,-0.061450,-0.030574,-0.040116,0.000670
NumOT,-0.007047,0.029183,1.000000,-0.001354,-0.012087,-0.003852,0.004209,0.086315,0.150996,0.052069,0.063894,0.037333,0.049701,0.117702,0.001131,-0.014027,-0.005425,0.003798,0.089983,0.146445,0.052321,0.058816,0.041808,0.042328,0.114739,-0.001447
A,0.009737,0.000967,-0.001354,1.000000,0.016481,0.003061,0.008674,0.008060,0.019199,0.015799,-0.025803,-0.010361,0.004834,-0.018350,0.003007,-0.017845,-0.003279,0.001182,-0.019000,-0.015249,-0.015893,-0.012271,-0.015455,-0.003525,0.005906,0.026964
FGP_A,0.009627,0.023299,-0.012087,0.016481,1.000000,0.530926,0.064705,-0.317734,0.150276,0.553796,-0.005246,0.033904,0.084231,-0.071488,-0.016835,-0.091070,-0.023645,0.006832,-0.010423,-0.643924,-0.101378,-0.026481,-0.026733,-0.343658,0.054859,0.487247
FGP3_A,-0.018803,0.021665,-0.003852,0.003061,0.530926,1.000000,0.054783,-0.211456,0.051728,0.418819,-0.015665,-0.030996,0.018917,-0.036263,-0.005377,-0.023163,-0.002112,0.011506,0.017140,-0.349624,-0.030629,-0.060904,-0.024983,-0.093288,-0.006833,0.313650
FTP_A,0.061249,0.051120,0.004209,0.008674,0.064705,0.054783,1.000000,-0.115769,0.020476,0.029689,-0.054903,-0.022284,-0.006524,-0.030155,0.004390,0.005727,0.005450,0.033042,-0.028845,-0.219841,-0.014567,-0.040531,-0.039034,-0.035724,0.018083,0.139325
OR_A,-0.229456,-0.057101,0.086315,0.008060,-0.317734,-0.211456,-0.115769,1.000000,0.037669,-0.005171,0.093060,0.106971,0.087267,0.138162,-0.013616,-0.000692,0.024931,-0.033489,0.084932,0.008331,0.024161,0.134179,0.060803,0.266348,0.160411,-0.019289
DR_A,0.096219,-0.039360,0.150996,0.019199,0.150276,0.051728,0.020476,0.037669,1.000000,0.201442,0.113724,-0.118497,0.267673,-0.025531,-0.013019,-0.644755,-0.355133,-0.221782,0.010035,0.032444,-0.249658,-0.206561,0.069441,-0.053891,0.137203,0.392306
AST_A,-0.031613,-0.031691,0.052069,0.015799,0.553796,0.418819,0.029689,-0.005171,0.201442,1.000000,-0.077757,0.129349,0.131957,-0.069488,-0.014457,-0.093647,-0.032524,-0.014429,0.017601,-0.245293,0.023556,0.094579,-0.028280,-0.141038,-0.017636,0.371480


### Few heavy correlation
- FGP_A - FGP3_A (0.53)
- FGP_A - AST_A (0.55)
- FGP_A - DR_B (negative)
- FGP3_A - AST_A
- TO_A - STL_B (quite high, 0.71)
- STL_A - TO_B (quite high, 0.71)
same for B

### Features
- Win %? O.o
- Plus_Minus